In [ ]:
import nltk
nltk.download('all')
import re
import string
import gensim 
import os
import PyPDF2
import fitz
from gensim.parsing.preprocessing import remove_stopwords
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from nltk import pos_tag,word_tokenize,ne_chunk
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas import DataFrame

from nltk.corpus import wordnet,stopwords
import spacy
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from flask import Flask, render_template, request, jsonify, send_from_directory



sample = open("Abstract.txt", "r") 
s = sample.read() 



app = Flask(__name__)

#-----------------------------------------------------------------------
UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure the upload folder exists
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
#-----------------------------------------------------------------------

def copy(filepath):
    global filepath_copy
    filepath_copy=filepath

def stem_sentence(sentence):
  words=word_tokenize(sentence)
  #lemmatizer = WordNetLemmatizer()
  
  
  stemmer = SnowballStemmer("english")
  new_words=[]
  for i in words:
    new_words.append(stemmer.stem(i))
    new_words.append(" ")
  return "".join(new_words)  
  


def clean_sentence(sentence, stopwords=True):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    
    
    
    if stopwords:
         sentence = remove_stopwords(sentence)
    
   
    
    return sentence


def get_cleaned_sentences(sents,stopwords=True):    
    
    cleaned_sentences=[]

    for i in sents:
        
        cleaning=clean_sentence(i,stopwords)
        cleaned=stem_sentence(cleaning)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences



def create_document_term_matrix(sen,vectorizer):
  doc_term_matrix=vectorizer.fit_transform(sen)
  return DataFrame(doc_term_matrix.toarray(), columns=vectorizer.get_feature_names_out())
  



def calculate_cosine_similarity(df_list,sentences,question):
  a=[]
  for i in range(len(df_list)-1):
    sim=1 - spatial.distance.cosine(df_list[i], question)
    t=(sim,sentences[i])
    a.append(t)
  a.sort(reverse=True)
  n=[]
  for i in range(3):
    n.append(a[i][1])
    #print("*",n[i])
   
    
  return n  


def questiontype( question):
        questiontags = ['WP','WDT','WP$','WRB']
        question_POS = pos_tag(word_tokenize(question.lower()))
        
        question_Tags=[]
        for token in question_POS:
            if token[1] in questiontags:
              question_Tags.append(token)
                
                
        if len(question_Tags)==1 and question_Tags[0][0]!= 'what' :
          return True
        else:
          return False  
        

from nltk.util import ngrams
def n_gram_similarity(question,n):
  q=list(ngrams(word_tokenize(question.lower()),1))
  a=0
  b=0
  c=0
  t=[]
  for i in q:
    if i in list(ngrams(word_tokenize(n[0].lower()),1)):
      a=a+1
  for i in q:
    if i in list(ngrams(word_tokenize(n[1].lower()),1)):
      b=b+1
  for i in q:
    if i in list(ngrams(word_tokenize(n[2].lower()),1)):
      c=c+1        
  d=max(a,b,c)
  if a == d:
    t.append(n[0])
  if b == d:
    t.append(n[1]) 
  if c ==d:
    t.append(n[2])
  print()  
  #print("Selected Sentence:",t[0])  
  return t  





def answertype(question,df_list,sentences):
  nlp = spacy.load('en_core_web_sm')

  if (questiontype(question)):
    t='DESCRIPTIVE'
    flag=0
    word=word_tokenize(question.lower())
  
    if 'who' in word:
      t='PERSON'
    elif 'where' in word:
      t='GPE'
    elif 'how' in word and 'many' in word and  'age' in word or 'duration' in word or 'long' in word or 'days'in word or 'years' in word or'months' in word:
      t='DATE' 
    elif 'how' in word and 'many' in word :
       t = 'CARDINAL'  
    elif 'when' in word  or 'age' in word or 'period' in word or 'duration' in word  or 'old' in word or 'long' in word:
      t='DATE'
    elif 'how' in word  and 'long' in word or 'often' or 'age' in word or 'years' in word:
      t='DATE' 
    elif 'what' in word and 'time' in word or 'duration' in word or 'period' in  'word'  :
      t='DATE' 
    i=len(df_list)-1  
    n=calculate_cosine_similarity(df_list, sentences,df_list[i])
    n=n_gram_similarity(question,n)
    #print("Most relevant sentence", n[0])
    #print("ANSWER TYPE:",t)
    key = n[0]
    spdoc = nlp(key)
    entity_type=[]
    for ent in spdoc.ents:
       if ent.label_ == t:
          entity_type.append(ent.text)
    if len(entity_type) == 1:
      #print("ANSWER TYPE:", t)
       print("ANSWER:", entity_type[0])
       return entity_type[0]  
    if len(entity_type) == 0:
      #print("ANSWER TYPE:", t) 
       print(n[0])
       return  n[0]
    if len(entity_type) > 1:
      #print("Answer Type:",t)  
      key_question = question
      q=[]
      spdoc = nlp(key_question)
      for ent in spdoc:
        if ent.pos_ == 'NOUN' or ent.pos_ =='ADJ' :
          q.append(ent.text)
  
      key_answer = n[0]
      a = []
      spd = nlp(key)
      for ent in spd:
        if ent.pos_ == 'NOUN'or ent.pos_ =='ADJ' :
          a.append(ent.text)
  #s=[sentence.index(i) for i in t]
      s=[]
      w=[]
      for i in entity_type:
       s.append(n[0].index(i))
      for i in range(len(s)):
        w.append(0)

    
      for i in q:
        try:
           factor= n[0].index(i)
           for j in range(len(s)):
              w[j]=w[j]+(abs(s[j]-factor))
        except:
           continue    
      m=min(w)
      u=[]
      for i in range(len(s)):
        if w[i] == m:
           #print(entity_type[i])
           u.append(entity_type[i])
      print("ANSWER:",u[0])   
      return u[0]

    
  else:
    t='DESCRIPTIVE'
    l=[]
    print("ANSWER TYPE:",t)
    i=len(df_list)-1  
    n=calculate_cosine_similarity(df_list, sentences,df_list[i])
    #n = n_gram_similarity(question, n)
    for j in n:
         l.append(j)
         print(j) 
        
    l="".join(l) 
    return l


@app.route("/")
def index():
    return render_template('base.html')


@app.route("/get", methods=["GET", "POST"])
def chat():
    
    doc = fitz.open(filepath_copy)
    text = ""

    # Iterate through each page
    for page in doc:
        text += page.get_text()

    doc.close()
    print("--------------------------F------------------------------")
    print(text)
    print("--------------------------F------------------------------")
    print(s)
    print("--------------------------F------------------------------")
    msg = request.form["msg"]
    input = msg
    #print(input)
    question=[input]

    #question=["What happens upon photolysis?"]
    for j in question:
    #j=TextBlob(j)
    #j=str(j.correct())
    
        qq=[]
        qp=[]
        que=sent_tokenize(j)
        
        qq.append(que)
        qp.append(j)
        questiontags = ['WP','WDT','WP$','WRB']
        question_POS = pos_tag(word_tokenize(j.lower()))
                
        question_Tags=[]
    for token in question_POS:
        if token[1] in questiontags:
            question_Tags.append(token)
    
    
    if len(question_Tags)>1:
        if ' and ' in j :
            pos=j.lower().find('and')
            qq=[]
            qp=[]
            qp.append(j[:pos])
            qp.append(j[pos+1:])
            qq.append(sent_tokenize(j[:pos])) 
            qq.append(sent_tokenize(j[pos+1:]))
            
        
    

    for k in range(len(qp)):   
        


        sentences=sent_tokenize(text)
        #q contains a list of cleaned sentence tokens of question
        q=get_cleaned_sentences(qq[k],stopwords=True)
        #preprocessed contains a list of cleaned sentence tokens of the reference text
        preprocessed=get_cleaned_sentences(sentences,stopwords=True)
    
        preprocessed.append(q[0])
        i=len(preprocessed)-1
        
    
        tfidf_vect=TfidfVectorizer()
        df=create_document_term_matrix(preprocessed,tfidf_vect) 
        df_list = df.values.tolist()
        
        x=answertype(qp[k],df_list,sentences)
        print("oooooooooooooo-----------------------------------oooooooooooooooo")
        print(x)
        
    return x

#------------------------------------------
@app.route('/uploader', methods=['POST'])
def uploader_file():
    if request.method == 'POST':
        f = request.files['file']
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], f.filename)
        print(filepath)
        copy(filepath)
        f.save(filepath)
        return render_template('chat.html', filename=f.filename)

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)
#-------------------------------------------



if __name__ == '__main__':
    app.run()


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/manasvivarma/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/manasvivarma/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/manasvivarma/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/manasvivarma/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/manasvivarma/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_

 * Serving Flask app '__main__'
 * Debug mode: off


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/manasvivarma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/manasvivarma/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Apr/2024 12:15:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2024 12:15:46] "POST /uploader HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2024 12:15:46] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Apr/2024 12:15:46] "GET /uploads/Untitled.pdf HTTP/1.1" 200 -


uploads/Untitled.pdf
--------------------------F------------------------------
A line graph is a unique graph which is commonly used in 
statistics. It represents the change in a quantity with respect to 
another quantity. For example, the price of different ﬂavours of 
chocolates varies, which we can represent with the help of this 
graph. This variation is usually plotted in a two-dimensional XY 
plane. If the relation including any two measures can be expressed 
utilizing a straight line in a graph, then such graphs are called linear 
graphs. Thus, the line graph is also called a linear graph. Here, we 
are going to discuss what a line graph is, its types, procedure to 
draw a line graph, and examples in detail.
A line graph or line chart or line plot is a graph that utilizes points 
and lines to represent change over time. It is a chart that shows a 
line joining several points or a line that shows the relation between 
the points. The graph represents quantitative data between two

127.0.0.1 - - [16/Apr/2024 12:16:05] "POST /get HTTP/1.1" 200 -


...

uploads/Abstract (1).pdf
--------------------------F------------------------------
Abstract
The molecules CF2Cl2 and CFCl3 are released into the atmosphere following their extensive use as
propellants for aerosol spray cans and in refrigeration. Since they are chemically inert and have low
solubility in water, these chlorofluoromethanes have very long atmospheric residence times and can be
detected throughout the troposphere in amounts roughly corresponding to the integrated world industrial
production to date. The most important sink for atmospheric CFCl3 and CF2Cl2 appears to be
photolytic dissociation in the stratosphere by ultraviolet radiation around 2000 Å. Upon photolysis the
chlorofluoromethanes release chlorine atoms, which initiate an extensive catalytic chain reaction
Cl + O3 → ClO + O2
ClO + O → Cl + O2
leading to the net destruction of O3 in the stratosphere. This chain reaction can be diverted through
reaction of ClO with NO, which interconnects the NOx and ClOx catalytic

127.0.0.1 - - [16/Apr/2024 12:29:14] "POST /get HTTP/1.1" 200 -


ANSWER TYPE: DESCRIPTIVE
Upon photolysis the
chlorofluoromethanes release chlorine atoms, which initiate an extensive catalytic chain reaction
Cl + O3 → ClO + O2
ClO + O → Cl + O2
leading to the net destruction of O3 in the stratosphere.
This chain reaction can be diverted through
reaction of ClO with NO, which interconnects the NOx and ClOx catalytic cycles.
This
depletion would increase up to 15 or 20% if the chlorofluoromethane injection were to continue
indefinitely at present rates.
oooooooooooooo-----------------------------------oooooooooooooooo
Upon photolysis the
chlorofluoromethanes release chlorine atoms, which initiate an extensive catalytic chain reaction
Cl + O3 → ClO + O2
ClO + O → Cl + O2
leading to the net destruction of O3 in the stratosphere.This chain reaction can be diverted through
reaction of ClO with NO, which interconnects the NOx and ClOx catalytic cycles.This
depletion would increase up to 15 or 20% if the chlorofluoromethane injection were to continue
indefi

127.0.0.1 - - [16/Apr/2024 12:29:31] "POST /get HTTP/1.1" 200 -



Furthermore, the calculations show that the full stratospheric effect of the
photodissociation of CF2Cl2 and CFCl3 is not immediately felt after their introduction at ground level
because of the delay required for upward diffusion to the 25- to 30-km level.
oooooooooooooo-----------------------------------oooooooooooooooo
Furthermore, the calculations show that the full stratospheric effect of the
photodissociation of CF2Cl2 and CFCl3 is not immediately felt after their introduction at ground level
because of the delay required for upward diffusion to the 25- to 30-km level.
